In [2]:
'''
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
'''

import html5lib
from bs4 import BeautifulSoup
from random import randint
import re
import os
import csv
import glob
from time import sleep, time

# Behind SOCKS5 Proxy

In [3]:
import requests
import socket
import socks

PORT = 1080

# no authentication
# socks.set_default_proxy(socks.SOCKS5, "localhost")

# with Authentication
socks.set_default_proxy(socks.SOCKS5, "proxy-nl.privateinternetaccess.com",PORT,True,"x6122622","A8yZzRo6GH")
socket.socket = socks.socksocket

# Load Data ID's

In [4]:
def csv_to_list(file):
    with open('{0}'.format(file), 'rt') as f:
        return [row[0] for row in csv.reader(f)]

# Pull Down Resumes from Site

In [5]:
def html_to_text(id_name):
    url = 'http://www.indeed.com{0}co=us/pdf'.format(id_name)
    
    ### SLEEP
    sleep(randint(1,5))
    ### SLEEP
    
    # request content from site
    try:
        html = requests.get(url, stream=True)
        
        data = html.text.encode('utf-8').decode('ascii', 'ignore')
        soup = BeautifulSoup(data, 'lxml')

        # get only resume content
        res = soup.find('div', {'id': 'resume'})
        res = " ".join(res.strings)

        # remove special characters
        res = re.sub(r'([-/*&%()@#$^])', ' ', res)

        cleaner = ['\xa0']
        for c in cleaner:
            res = res.replace(c,'')

        # remove multiple white spaces
        res = re.sub(' +',' ', res)
    except:
        res = 'FAIL'
        
    return res

In [10]:
def download_text(title):
    csv_file = 'data/{0}.csv'.format(title)
    
    if os.path.isfile(csv_file):
        
        # loop through name id's
        name_ids = csv_to_list(csv_file)
        for i,name in enumerate(name_ids):
            # start new timer
            start_time = time()
            
            # CALL TO GET RESUME FROM SITE
            text_res = html_to_text(name)
            if text_res != 'FAIL':
                ofile = 'data/pdfs/{0}_{1}.txt'.format(title, i)

                # delete old version
                if os.path.exists(ofile):
                    os.remove(ofile)

                # write to file
                with open(ofile, 'w') as f:
                        f.write(text_res)

                # log results to file to show progress/status
                length = len(name_ids) - i
                with open('data/logfile.txt', 'a') as f:
                    elapsed_time = time() - start_time
                    log = '{0}: time {1}s\t remaining {2} \n'.format(title, str(round(elapsed_time,3)), length)
                    f.write(log)
            else:
                with open('data/stopped.txt', 'a') as f:
                    elapsed_time = time() - start_time
                    log = '{0}: time {1}s\t remaining {2} (list id {3}) \n'.format(title, 
                                                                str(round(elapsed_time,3)), length, i)
                    f.write(log)

# List of Resume Link csv Files

In [11]:
def all_titles():
    all_files = glob.glob('data/*.csv')
    copy_files = glob.glob('data/* copy.csv')
    full_files = list(set(all_files) - set(copy_files))
    titles = [re.search('data\/(.*?)\.csv',x).group(1) for x in full_files]
    return titles

# Master Download Loop

In [12]:
all_titles()[1:]

['scientist', 'analytics', 'big_data', 'data_analysis', 'data']

In [13]:
# delete old version
if os.path.exists('data/logfile.txt'):
    os.remove('data/logfile.txt')

# loop through all the titles of the id files downloaded
for file in all_titles()[1:]:
    download_text(file)

# Log In to Indeed